In [1]:
import sys
sys.path.append('..')

from retriever.core.rags import SimpleRag

simple_rag = SimpleRag("exercises")

d:\Z\Conspects\_autumn\PAK\Bilyaletov\rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Z\Conspects\_autumn\PAK\Bilyaletov\rag\.venv\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [10]:
from pydantic import BaseModel

class RAGTest(BaseModel):
    query: str
    expected_tags: list[str]


In [11]:


rag_test = [
    RAGTest(query="Как делать жим лёжа", expected_tags=['Жим лёжа', 'technique']),
    RAGTest(query="Как правильно стоять в планке", expected_tags=['Планка', 'technique']),
    RAGTest(query="Альтернативы для приседания со штангой", expected_tags=['Приседания со штангой', 'alternatives']),
    RAGTest(query="Как мышцы работают в подтягиваниях", expected_tags=['Подтягивания', 'muscles']),
]

In [12]:
points = simple_rag.request("Техника выполнения жима лёжа")

for point in points:
    payload = point.payload
    del payload["text"]
    print(payload)

def hit5_test():
    total_absolute_hits = []
    total_hits = {"exercise": 0, "type": 0}
    for test in rag_test:
        hits = {expected_tag:0 for expected_tag in test.expected_tags}
        absolute_hit_position = -1
        results = simple_rag.request(test.query)
        for i in range(len(results)):
            result = results[i]
            payload = result.payload
            del payload['text']
            print(payload)
            tags = payload['tags']
            
            for expected_tag in test.expected_tags:
                if expected_tag in payload['tags']:
                    hits[expected_tag] += 1 
            if all([expected_tag in payload['tags'] for expected_tag in test.expected_tags]):
                absolute_hit_position = i
        print(hits)

        total_hits["exercise"] += hits[test.expected_tags[0]]
        total_hits["type"] += hits[test.expected_tags[1]]
        if absolute_hit_position != -1:
            total_absolute_hits.append(absolute_hit_position)
    
    print(total_hits)
    print(total_absolute_hits)


hit5_test()



{'source': 12, 'tags': ['Планка', 'technique']}
{'source': 13, 'tags': ['Подтягивания', 'technique']}
{'source': 2, 'tags': ['Жим лёжа', 'technique']}
{'source': 4, 'tags': ['Становая тяга', 'technique']}
{'source': 15, 'tags': ['Русские скручивания', 'technique']}
{'source': 12, 'tags': ['Планка', 'technique']}
{'source': 13, 'tags': ['Подтягивания', 'technique']}
{'source': 5, 'tags': ['Ягодичный мост', 'alternatives']}
{'source': 2, 'tags': ['Жим лёжа', 'alternatives']}
{'source': 6, 'tags': ['Подъём ног с наклоном корпуса', 'alternatives']}
{'Жим лёжа': 1, 'technique': 2}
{'source': 16, 'tags': ['Боковая планка', 'alternatives']}
{'source': 11, 'tags': ['Тяга гантели одной рукой', 'alternatives']}
{'source': 12, 'tags': ['Планка', 'mistakes']}
{'source': 3, 'tags': ['Болгарский сплит-присед', 'technique']}
{'source': 7, 'tags': ['Тяга верхнего блока к груди', 'alternatives']}
{'Планка': 1, 'technique': 1}
{'source': 1, 'tags': ['Приседания со штангой', 'technique']}
{'source': 3, '